In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('/kaggle/input/ioai-contest-1/train.csv')
sample_submission = pd.read_csv('/kaggle/input/ioai-contest-1/sample_submission.csv')

In [2]:
train

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Company_ID,Product_ID,Target,Date,cat_0,cat_1,cat_2,cat_3,cat_4,cat_5,...,num_9,num_10,num_11,num_12,num_13,num_14,num_15,num_16,num_17,num_18
0,3,10395,1,2021-09-12,1,3689,2,2,0.0,1,...,NaN,NaN,1.0,1.0,136.800,136.800000,136.80,152.95,136.80,136.80
1,3,11303,10,2023-02-12,3,1736,2,2,2.0,3,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2499,2,2020-05-24,1,3572,1,2,0.0,4,...,NaN,2.493151,3.0,2.0,411.939,677.394667,830.28,908.60,830.28,830.28
3,3,3118,10,2019-07-12,3,1435,2,2,0.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,13044,6,2019-02-29,1,1263,0,2,0.0,3,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5310945,2,13577,4,2020-03-13,1,7875,1,2,0.0,2,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
5310946,2,1105,1,2022-01-05,1,6299,1,2,0.0,2,...,0.0,NaN,3.0,2.0,77.148,111.096000,151.74,104.28,104.40,104.40
5310947,3,3896,1,2020-09-17,1,3817,2,2,2.0,2,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
5310948,1,3097,1,2023-11-01,1,7873,0,2,2.0,3,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
train['month'] = train['Date'].apply(lambda x: '-'.join(x.split('-')[:2]))

In [4]:
grs = train.groupby(['Company_ID', 'Product_ID', 'month'], as_index=False)['Target'].agg([('Target', 'sum')])

In [5]:
grs

,Company_ID,Product_ID,month,Target
0,0,1,2019-05,2
1,0,1,2019-06,13
2,0,1,2019-07,1
3,0,1,2019-09,30
4,0,1,2019-11,1
...,...,...,...,...
862331,3,14668,2021-03,1
862332,3,14668,2021-04,1
862333,3,14668,2021-05,1
862334,3,14668,2021-06,1


In [6]:
sample_submission['Company_ID'] = sample_submission['Id'].apply(lambda x: int(x.split('_')[0]))
sample_submission['Product_ID'] = sample_submission['Id'].apply(lambda x: int(x.split('_')[1]))

predict = grs.groupby(['Company_ID', 'Product_ID'])['Target'].agg([
    ('sum_3m', 'sum'),
    ('mean_3m', 'mean'),
    ('median_3m', 'median')
]).reset_index()

predict['Target'] = predict['median_3m'].apply(round)

In [7]:
predict

,Company_ID,Product_ID,sum_3m,mean_3m,median_3m,Target
0,0,1,413,8.098039,7.0,7
1,0,2,24,1.263158,1.0,1
2,0,3,61,2.652174,2.0,2
3,0,5,70,1.794872,1.0,1
4,0,6,161,3.833333,3.0,3
...,...,...,...,...,...,...
37947,3,14664,2,1.000000,1.0,1
37948,3,14665,4210,120.285714,100.0,100
37949,3,14666,3,3.000000,3.0,3
37950,3,14667,3,1.000000,1.0,1


In [8]:
submission = sample_submission.merge(predict, on=['Company_ID', 'Product_ID'], how='left')

In [9]:
submission['Target'] = submission['Target_y'].fillna(0)

In [10]:
submission = submission[['Id', 'Target']]

In [11]:
submission.to_csv('submission.csv', index=False)